In [1]:
import pandas as pd
import os
import re
from tqdm import tqdm_notebook as tqdm
import numpy as np

data_dirpath = '/usr2/mamille2/tumblr/data/'

# Measure proportion of hegemonic labels

## Define set of hegemonic labels

In [5]:
hegemonic_labels = {
    'gender': [
        r'\bcis',
    ],
    'sexual orientation': [
        'straight', r'\bcishet',
    ],
    'pronouns': [ # Should be better about any type of delimiter or no delimiter with these
        'she/her',
        'she her',
        'he/him',
        'he him',
    ],
    'ethnicity/nationality': [
        'white', 'caucasian',
    ],
}

opp_hegemonic_labels = {
    'gender': [
        'trans', 'ftm', 'mtf',
        'nonbinary', r'non-binary', r'non binary', '\bnb\b', '\bbigender', r'\bagender', r'neutrois', 
        r'genderfluid', r'gender-fluid',
        r'lgbt', r'\bqueer\b',
    ],
    'sexual orientation': [
        r'gay', 'r\bhomo\b',
        r'lesbian',
        r'bisexual', r'\bbi\b', r'bisexual', r'pansexual', r'\bpan\b', r'\bwlw\b', r'\bmlm\b',
        r'lgbt', r'\bqueer\b',
        r'\bace\b', r'\basexual', r'aro-ace', r'aro/ace',
        r'demisexual',
    ],
    'pronouns': [ # Should be better about any type of delimiter or no delimiter with these
        r'(?:\W|\b)they(?:\W|\b)', r'(?:\W|\b)them(?:\W|\b)',
        r'(?:\W|\b)xe(?:\W|\b)', r'(?:\W|\b)xem(?:\W|\b)',
        r'theythem',
    ],
    'ethnicity/nationality': [
        r'black', r'african(-| )american', r'afro(-| )american',
        r'latin', r'afrolatin',
        r'cuban', r'mexican',
        r'asian', r'chinese', r'korean', r'japanese', r'indian',
    ],
}

hegemonic_excl_terms = {
    'gender': [
        r"isn't a cishet",
        r'if ur a cishet',
        r"don't call me: cishet",
        r"cishet aces",
        r"anti-cishet",
        r"cishets begone",
        r"cishet scum",
        r"hate cishets",
        r"cishet sims",
        r"isntcishet",
        r"cishets",
        r"you are white or cishet",
        r"cishet men",
        r"cishet males",
        r"cishet guys",
        r"terf/twerf/cishet/man",
        r"if cishet",
        r"not cishet",
        r"isn't a cishet",
        r'if ur a cishet',
    ],
    'pronouns': [
        'they/them',
    ],
    'ethnicity/nationality': [
        r'white lies',
        r'snow white',
        r'great white',
    ],
}
hegemonic_excl_terms['sexual orientation'] = hegemonic_excl_terms['gender']

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 1000)

In [ ]:
# Load annotated data
# data = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100_100posts.pkl'))
data = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100.pkl'))
print(len(data))
data.columns

In [ ]:
# Basic stats
cats = ['age', 'location', 'gender', 'sexual orientation', 'pronouns', 
        'personality type', 'ethnicity/nationality', 'relationship status', 'roleplay',
        'fandoms', 'interests', 'weight',
        'gender/sexuality', 'roleplay/fandoms']

outlines = []

for col in cats:
    annotated = sum(data[col])
    outlines.append([col, annotated, annotated/len(data) * 100])
#     print(f"{col}: {annotated}\t{annotated/len(data): .1%}")

# No annotations
any_annote = sum([any(line) for line in list(zip(*[data[col] for col in cats]))])
no_annote = len(data) - any_annote
# print()
# print(f'# with no annotations: {no_annote}\t{no_annote/len(data): .1%}')
outlines.append(['none', no_annote, no_annote/len(data) * 100])

table = pd.DataFrame(outlines, columns=['category', 'n_instances', '% descriptions'])\
        .sort_values('% descriptions', ascending=False).reset_index(drop=True)
table

## Annotate hegemonic, opposite labels

In [ ]:
# Annotate for hegemonic, opposite labels
for cat, terms in sorted(hegemonic_labels.items()):
    print(cat)
#     data[f'{cat}_hegemonic_present'] = data['parsed_blog_description'].map(lambda x: any(re.search(t, x) for t in terms) and \
#                                                                           not any(re.search(e, x) for e in hegemonic_excl_terms[cat]))
    data[f'{cat}_hegemonic_present'] = list(map(lambda x: any(re.search(t, x) for t in terms) and not any(re.search(e, x) for e in hegemonic_excl_terms[cat]),
                                                                                                          data['parsed_blog_description']))
    
print()
for cat, terms in sorted(opp_hegemonic_labels.items()):
    print(cat)
#     data[f'{cat}_opposite_present'] = data['parsed_blog_description'].map(lambda x: any(re.search(t, x) for t in terms))
    data[f'{cat}_opposite_present'] = list(map(lambda x: any(re.search(t, x) for t in terms), tqdm(data['parsed_blog_description'].tolist())))

In [ ]:
# Combine gender, sexual orientation and pronouns to annonate for gender/sexuality/hegemonic
data['gender/sexuality_opposite_present'] = [g or s or p for (g,s,p) in zip(*[data[f'{cat}_opposite_present'] for cat in ['gender', 'sexual orientation', 'pronouns']])]
print(sum(data['gender/sexuality_opposite_present']))
data['gender/sexuality_hegemonic_present'] = [g or s or p for (g,s,p) in zip(*[data[f'{cat}_hegemonic_present'] for cat in ['gender', 'sexual orientation', 'pronouns']])]
print(sum(data['gender/sexuality_hegemonic_present']))

In [ ]:
# Save hegemonic annotations
data.to_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100.pkl'))

In [ ]:
# Count hegemonic labels
hegemonic_count = {}
cats = list(hegemonic_labels.keys()) + ['gender/sexuality']
for cat in cats:
    hegemonic_count[cat] = len(data[(data[cat]==True) & (data[f'{cat}_hegemonic_present'] == True)])
    
# Count opposite labels
opp_count = {}
for cat in cats:
    opp_count[cat] = len(data[(data[cat]==True) & (data[f'{cat}_opposite_present'] == True)])

# Build table of results
outlines = []
for cat in cats:
    given_count = table.loc[table['category']==cat, 'n_instances'].item()
    outlines.append([cat, given_count, hegemonic_count[cat], hegemonic_count[cat]/given_count * 100,
                    opp_count[cat], opp_count[cat]/given_count * 100])
    
results = pd.DataFrame(outlines, columns=['category', 'given number', 'hegemonic number', 'hegemonic %', 
                                         'opposite number', 'opposite %'])
results

## 1-time

In [ ]:
# Check for certain terms
# term = r'\bhet\b'
term = r'\basdfalskdjf ;alkdjs f;lakjsa\b'
term_rows = data.loc[data['parsed_blog_description'].map(lambda x: True if re.search(term, x) is not None else False), ['parsed_blog_description', 'gender', 'sexual orientation']]
print(len(term_rows))
term_rows

In [ ]:
min(term_rows['gender'])

# See which factors are associated with giving a hegemonic label

## Count how many hegemonic labels are given w and wo non-hegemonic labels

In [2]:
# Load data
data = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100.pkl'))
print(len(data))
data.columns

5226750


Index(['tumblog_id', 'activity_time_epoch', 'tumblr_blog_name',
       'tumblr_blog_title', 'tumblr_blog_description', 'tumblr_blog_url',
       'tumblr_blog_theme', 'is_group_blog', 'is_primary', 'is_private',
       'created_time_epoch', 'updated_time_epoch', 'timezone', 'language',
       'blog_classifier', 'generated_date', 'parsed_blog_description',
       'age_terms', 'age', 'ethnicity/nationality_terms',
       'ethnicity/nationality', 'fandoms_terms', 'fandoms', 'gender_terms',
       'gender', 'gender/sexuality_terms', 'gender/sexuality',
       'interests_terms', 'interests', 'location_terms', 'location',
       'personality type_terms', 'personality type', 'pronouns_terms',
       'pronouns', 'relationship status_terms', 'relationship status',
       'roleplay_terms', 'roleplay', 'roleplay/fandoms_terms',
       'roleplay/fandoms', 'sexual orientation_terms', 'sexual orientation',
       'weight_terms', 'weight', 'ethnicity/nationality_hegemonic_present',
       'gender_hege

In [3]:
hegemonic_cols = [col for col in data.columns.tolist() if 'hegemonic' in col]
hegemonic_cols = [col for col in hegemonic_cols if col != 'gender/sexuality_hegemonic_present']
print(hegemonic_cols)

opposite_cols = [col for col in data.columns.tolist() if 'opposite' in col]
opposite_cols = [col for col in opposite_cols if col != 'gender/sexuality_opposite_present']
print(opposite_cols)

['ethnicity/nationality_hegemonic_present', 'gender_hegemonic_present', 'pronouns_hegemonic_present', 'sexual orientation_hegemonic_present']
['ethnicity/nationality_opposite_present', 'gender_opposite_present', 'pronouns_opposite_present', 'sexual orientation_opposite_present']


In [5]:
mask = [any(t) for t in zip(*[data[col] for col in opposite_cols])]
opp = data[mask]
count_opp = len(opp)
count_opp

In [9]:
count_nonopp = len(data) - count_opp
count_nonopp

4949845

In [10]:
len(all_hegemonic)/count_nonopp

0.011584201121449257

In [7]:
mask = [any(t) for t in zip(*[data[col] for col in hegemonic_cols])]
any_hegemonic = data[mask]
len(any_hegemonic)

81256

In [8]:
mask = [not any(t) for t in zip(*[any_hegemonic[col] for col in opposite_cols])]
all_hegemonic = any_hegemonic[mask]
len(all_hegemonic)

57340

In [ ]:
len(all_hegemonic)/len(any_hegemonic)